In [ ]:
import dspy
import random
import os

In [ ]:
LOCAL=True

## Learn DSPy fundamentals through conversation!

### This notebook demonstrates:

* DSPy Modules (conversation interface)
* DSPy Signatures (conversation interface)
* DSPy ReAct (tool usage & agent reasoning)
* Multi-Agent Orchestration (intelligent routing)
* DSPy History (context management)

In [ ]:
# Step 1: Configure DSPy with your LLM
if not LOCAL:
    MODEL='openai/gpt-5-nano'
    API_KEY=os.getenv('OPENAI_API_KEY')
    LLM = dspy.LM(model=MODEL, temperature=1.0, max_tokens=16000, api_key=API_KEY)
else:
    MODEL='lm_studio/qwen/qwen3-coder-30b'
    API_KEY='local'
    LLM = dspy.LM(model=MODEL, api_base="http://192.168.0.49:1234/v1/", api_key=API_KEY)

dspy.configure(lm=LLM)

Say hi! Real basic interface, to make sure we are connected to the LLM

In [ ]:
LLM("Hi there, can you send a greeting and a quick introduction of yourself!?")

## Modules and Signatures.

### Module is a "method of thinking".

In the background each module uses a slightly different prompting technique.
There are a few modules available.

* **Predict**: Basic predictor. Does not modify the signature. Handles the key forms of learning.
* **ChainOfThought**: Asks the LM to think step-by-step before committing to the signature's response.
* **ReAct**: An agent that can use tools to implement the given signature.
* **ProgramOfThought**: Asks the LM to output code, whose execution results will dictate the response.
* **MultiChainComparison**: Can compare multiple outputs from ChainOfThought to produce a final prediction.
* [More on the dspy docs](https://dspy.ai/learn/programming/modules/#what-other-dspy-modules-are-there-how-can-i-use-them)

### Signature is the input | output and extra contextual instructions to the LLM

This also affects the prompt sent to the LLM, keep it loose do not try to optimise keywords.

In [ ]:
qa = dspy.ChainOfThought(
    dspy.Signature('question -> answer')
)

question="What Year was The Eiffel Tower built?"

response = qa(question=question)

print(f'{MODEL}:', response.answer,'\n')
print(response.reasoning)

In [ ]:
what_month = dspy.Predict(
    dspy.Signature('description_of_a_month:str -> month:str')
)


strings = [
    "The first month of the year",
    "The month of leaves",
    "The month with the least amount of days",
    "The month of spring begins",
    "The month after April",
    "The one that has easter",
    "The month of summer begins",
    "The month of autumn begins",
    "The month of winter begins",
    "The one that has haloween",
    "The month of snow",
    'The third in the calendar'
]
for describe_a_month in strings:
    print(describe_a_month)

    response = what_month(description_of_a_month=describe_a_month)

    print('   *',response.month)

In [ ]:
text = "Apple Inc. Announced its latest iPhone 17 today. The CEO, Tim Cook, highlighted its new features in a press release."

structuriser = dspy.Predict("text -> title, headings: list[str], entities_and_metadata: list[dict[str, str]]")
response = structuriser(text=text)

print(response.title)
print(response.headings)
print(response.entities_and_metadata)

In [ ]:
print(f'There are {len(LLM.history)} items in the history\n')  

print(LLM.history[-1].keys(),'\n')  # access the last call to the LM, with all metadata

for item in LLM.history[-1:]:  
    for key, value in item.items():  
        print(f'{key}: {value}')  
    print('\n')


In [ ]:
toxicity = dspy.Predict(
    dspy.Signature(
        "comment:str -> toxic: bool",
        instructions="Mark as 'toxic' if the comment includes insults, harassment, or sarcastic derogatory remarks.",
    )
)

comments = [
    "you are ugly.",
    "What a nice guy!",
    "I hate you so much",
    "Great job on that project!",
    "You're worthless",
    "Thank you for your help",
    "Stop being so stupid",
    "You did an amazing work",
    "I can't stand you",
    "I appreciate your effort"
]

for comment in comments:
    print(comment,'\n  Toxic?:', toxicity(comment=comment).toxic)

In [ ]:
sentences = [
    "I'm so frustrated with this terrible service and endless waiting times.",
    "The weather is disappointing, but at least we have indoor activities.",
    "The meeting was scheduled for 3 PM, which is standard for our team.",
    "I'm feeling better after getting some rest and fresh air.",
    "What an amazing day filled with wonderful surprises and joyful moments!",
    "This product is absolutely horrible and waste of money.",
    "The customer support was very helpful and solved my issue quickly.",
    "I hate this movie, it's boring and poorly acted.",
    "The restaurant had excellent food and great atmosphere.",
    "This software is confusing and difficult to use."
]

sentiment_analysis = dspy.Predict(
    dspy.Signature(
        'sentence -> sentiment: int',
        instructions= "Give me a value from 1-10 indicating the sentiment, 1 being very negative and 10 being overwhelmingly positive",
        )
    )

for sentence in sentences:
    print(sentence)
    print(f'{sentiment_analysis(sentence=sentence).sentiment} out of 10\n')

In [ ]:
short_story = """The rain hammered against the cracked pavement as Marcus clutched his worn canvas bag, his weathered hands trembling slightly from the cold. He had been selling hot coffee and bagels on this corner for fifteen years, ever since his wife passed away, but tonight felt different—perhaps because the steady stream of commuters had thinned to just a few desperate souls seeking warmth. His small cart, tucked between the towering glass buildings, cast a faint glow through the misty darkness, and he watched as an elderly woman with a umbrella approached, her face illuminated by the yellow light. She purchased his last cup of coffee, and as she walked away, Marcus smiled, knowing that even in the rain, someone was still finding comfort in his simple offering.
The city's endless rhythm continued around him, but for this moment, he felt like he was making a difference. The rain had been falling for hours now, turning the streets into rivers of gray water that reflected the neon signs of the nearby restaurants and stores. His cart, though modest, had become a familiar presence on this corner, a small oasis in the concrete jungle where people could pause, warm themselves, and perhaps find a moment of human connection in their hurried lives.
As he wiped the condensation from his coffee cup, Marcus noticed a young man huddled under an overpass nearby, his jacket soaked through. The boy's eyes met Marcus's, and for a brief second, there was something in that look—hope, maybe desperation, or simply recognition of shared struggle. Marcus reached into his bag and pulled out another cup of coffee, offering it to the boy without hesitation. The young man accepted it with a quiet "thank you," and Marcus felt that familiar warmth spreading through his chest, the kind that only comes from knowing you've made someone's night just a little bit better.
The rain continued its relentless assault, but Marcus was no longer alone in the darkness. He had become more than just a vendor; he had become a small beacon of humanity in a city that often forgot to look up and notice the people who were trying to keep it all together, one cup of coffee at a time.
"""
print(f'original length (characters): {len(short_story)}')

summarize = dspy.ChainOfThought(
    dspy.Signature('story -> summary')
)
response = summarize(story=short_story)
print(f'summarised length (characters): {len(response.summary)}')

print(response.summary)
# print("Reasoning:", response.reasoning)

# Agents and Tools

In [ ]:
def roll_dice(sides: int = 6) -> str:
    """Roll a die with specified number of sides (default 6)."""
    result = random.randint(1, sides)
    return result

def flip_coin() -> str:
    """Flip a coin and return heads or tails."""
    return random.choice(["Heads", "Tails"])

def pick_random_card(num_cards: int = 1) -> str:
    """
    Pick a specified number of random cards from a standard 52-card deck.

    Args:
        num_cards (int, default 1): The number of cards to randomly select from the deck

    Returns:
        list: A list of randomly selected card strings in the format "Rank of Suit"

    Example:
        >>> pick_random_card(3)
        ['Ace of Hearts', '7 of Spades', 'King of Diamonds']
    """

    suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
    ranks = ['Ace', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King']
    deck = [f"{rank} of {suit}" for suit in suits for rank in ranks]
    return random.sample(deck, num_cards)

In [ ]:
react_agent = dspy.ReAct(
    signature="question -> answer",
    tools=[pick_random_card, roll_dice, flip_coin]
)

In [ ]:
result = react_agent(question="lets play a card game, deal me 5 cards")
print(result.answer)
print("Tool calls made:", result.trajectory)

In [ ]:
result = react_agent(question="Pick 3 random cards, if any of them are a face card, roll a D20")
print(result.answer)
print("Tool calls made:", result.trajectory)

# Multi-Agent Orchestration

In [ ]:
# Games Expert Agent
class GamesExpertSignature(dspy.Signature):
    """You are a fun and enthusiastic game master AI. When handling games like dice rolls 
    or coin flips, be playful and engaging. Add excitement and personality to the results. 
    Make it fun!"""
    question: str = dspy.InputField(desc="Game or random activity request")
    answer: str = dspy.OutputField(desc="Fun and engaging game result")

games_expert_agent = dspy.ReAct(
    signature=GamesExpertSignature,
    tools=[pick_random_card, roll_dice, flip_coin]
)

def consult_games_expert(question: str) -> str:
    """Use this expert when the user wants to play games, roll dice, flip coins, make random choices, 
    or needs help with any luck-based or random activities. This expert makes it fun!"""
    result = games_expert_agent(question=question)
    return result.answer

In [ ]:
# Step 5: Create the Orchestrator Agent
class OrchestratorSignature(dspy.Signature):
    """You are a helpful AI assistant orchestrator. Your job is to understand what the user needs 
    and route their question to the appropriate expert agent. Read the tool descriptions carefully 
    to decide which expert can best help. If the question doesn't need a specialized expert, 
    answer it yourself using general knowledge."""
    question: str = dspy.InputField()
    history: dspy.History = dspy.InputField()
    answer: str = dspy.OutputField()

orchestrator = dspy.ReAct(
    signature=OrchestratorSignature,
    tools=[consult_games_expert],
    max_iters=10
    )

In [ ]:
history = dspy.History(messages=[])

question='Hi there, can you tell me your name and then flip a coin?'
result = orchestrator(question=question, history=history)
answer = result.answer

history.messages.append({"question": question, **result})
print(answer)
print(result.trajectory)

In [ ]:
# Check if any expert agents were called
expert_calls = []

for key, value in result.trajectory.items():
    if 'tool_name' in key and value != 'finish':
        expert_calls.append(str(value))

if expert_calls:
    print(f"🔀 Routed to: {', '.join(set(expert_calls))}")
else:
    print("💬 Handled directly by orchestrator")
